In [10]:
import pandas as pd
from datetime import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
import scipy.spatial.distance as ssd
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
os.chdir(r"C:\Users\A4023862\Documents\GitHub")
from path_segmentation.functions import *
import warnings
warnings.filterwarnings("ignore")

def clean_cols(df):
    import re
    
    cols=list(df.columns.values)
    
    # Lowercase everything
    cols=list(map(lambda x: x.lower(), cols))
    
    # Remove special characters 
    cols = [re.sub(r'[^a-zA-Z0-9]','_',string) for string in cols]
    
    # Rename colums
    df.columns = cols

    return df

In [13]:
df=pd.read_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\data\emr\df.csv")

In [14]:
cols=['interaction_id', 'id', 'rec_date', 'month_id', 'flag', 'value', 'dob', 'ethnicity', 'race', 'state', 'year', 'age', 'row', 'max','age_grp']
df.columns = cols
df.head()

,interaction_id,id,rec_date,month_id,flag,value,dob,ethnicity,race,state,year,age,row,max,age_grp
0,614619_2020-05-26_0035_37,616541,2020-05-26 18:30:12,202005,bmi,45.0,1974,NaN,AFRICAN AMERICAN,NY,2020,46,35,37,46-50
1,614619_2020-05-26_0036_37,616541,2020-05-26 18:56:49,202005,hyst_oophorectomy,1.0,1974,NaN,AFRICAN AMERICAN,NY,2020,46,36,37,46-50
2,614619_2020-05-26_0037_37,616541,2020-05-26 18:56:49,202005,hysterectomy,1.0,1974,NaN,AFRICAN AMERICAN,NY,2020,46,37,37,46-50
3,621586_2014-02-21_0006_40,623508,2014-02-21 05:00:00,201402,headache_migraine,1.0,1958,NaN,UNKNOWN,NY,2014,56,6,40,56-60
4,621586_2014-02-21_0007_40,623508,2014-02-21 05:00:00,201402,headache_migraine_freq,2.0,1958,NaN,UNKNOWN,NY,2014,56,7,40,56-60


# Data Prep

In [15]:
values='value'
index=['id', 'rec_date', 'age', 'age_grp', 'race', 'state']
columns='flag'
aggfunc=max

df_pivot = data_pivot(df,values,index,columns,aggfunc)
df_pivot.head()

,id,rec_date,age,age_grp,race,state,alcohol_consumption,amenorrhea,anxiety,bc_implant,...,smoker,stress_incontinence,uni_oophorectomy,urge_incontinence,uti,vaginal_dryness,vaginal_dryness_freq,vaginal_dryness_rx,vaginal_dryness_sev,weight_gain
0,612271,2015-11-04 19:26:00,46,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,612271,2015-11-04 19:56:44,46,46-50,CAUCASIAN,NY,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,612271,2015-11-10 14:41:28,46,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,612271,2016-01-29 18:44:02,47,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,612271,2016-03-07 19:06:08,47,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
q=df_pivot[df_pivot['last_period']==1]
q=q[['id','rec_date']]
q=q.groupby(['id']).agg(['min']).reset_index()
q.columns = ['id', 'min_last_period_date']

w = pd.merge(df_pivot, q, how="left", on="id")
w['post_meno']=np.where(w['rec_date']>= w['min_last_period_date'],1,0)
w.head()

,id,rec_date,age,age_grp,race,state,alcohol_consumption,amenorrhea,anxiety,bc_implant,...,uni_oophorectomy,urge_incontinence,uti,vaginal_dryness,vaginal_dryness_freq,vaginal_dryness_rx,vaginal_dryness_sev,weight_gain,min_last_period_date,post_meno
0,612271,2015-11-04 19:26:00,46,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,0
1,612271,2015-11-04 19:56:44,46,46-50,CAUCASIAN,NY,0,0,1,0,...,0,0,0,0,0,0,0,0,NaN,0
2,612271,2015-11-10 14:41:28,46,46-50,CAUCASIAN,NY,0,0,0,0,...,1,0,0,0,0,0,0,0,NaN,0
3,612271,2016-01-29 18:44:02,47,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,0
4,612271,2016-03-07 19:06:08,47,46-50,CAUCASIAN,NY,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,0


In [29]:
len(np.unique(w['id']))

96929

In [ ]:
w.columns

In [31]:
symp_list=['id', 'rec_date', 'age',
       'alcohol_consumption', 'amenorrhea', 'bc_implant',
       'bc_injection', 'bc_oral', 'bc_other', 'bi_oophorectomy',
       'birth_control',
       'dec_libido','dry_skin', 'dyspareunia',
       'fatigue', 'hair_loss', 'headache_migraine',
        'hot_flash','hot_flash_rx',
       'hyst_oophorectomy', 'hysterectomy',
       'last_period', 'memory_lapse', 'menopause',
       'night_sweats', 'night_sweats_rx','oab_incontinence', 'sleep_disturbance', 'smoker',
       'stress_incontinence', 'uni_oophorectomy', 'urge_incontinence', 'uti',
       'vaginal_dryness', 'weight_gain',
       'post_meno']

train=w[symp_list]

In [37]:
# Filter patients with surgical menopause, cancer
w1=w[w['bi_oophorectomy']==1]
w2=w[w['hysterectomy']==1]
w3=w[w['uni_oophorectomy']==1]
w4=w[(w['cancer']==1) | (w['breast_cancer']==1)]
w5 = pd.concat([w1,w2,w3, w4],ignore_index=True)

train=train[~train.id.isin(w5['id'])]

#seperate pre and post meno observations
train1=train[train['post_meno']==0].reset_index(drop=True)
train2=train[train['post_meno']==1].reset_index(drop=True)

print('# of patients with ooph, hyst or cancer: %d' % w5.id.nunique())
print('# of patients overall: %d' % w.id.nunique())
print('# of patients final: %d' % train.id.nunique())
print('# of patients pre: %d' % train1.id.nunique())
print('# of patients post: %d' % train2.id.nunique())

# of patients with ooph, hyst or cancer: 24504
# of patients overall: 96929
# of patients final: 72425
# of patients pre: 64826
# of patients post: 13189


In [38]:
find_nulls(train)

No Nulls found


In [39]:
#sample 500 patients
train1_sampled = sample_patients(train1,500, 'id')
train2_sampled = sample_patients(train2,500'id')

AttributeError: 'DataFrame' object has no attribute 'patient_id'